<a href="https://colab.research.google.com/github/SanikaAlmeida/LSTM-FakeNews/blob/main/LSTM_FakeNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [ ]:
#Steps:
# 1.Dataset cleaning
# 2.Separate X and y
# 3.Cleaning the data- Stemming,Stopwords
# 4.One hot repr
# 5.Same sentence length
# 6.LSTM model with embedding and dropout layer

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('train.csv')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.shape

(20800, 5)

In [ ]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [ ]:
###Drop Nan Values
df=df.dropna()
##Cannot fill as its text data and random/avg or any other filling wont work

In [ ]:
df.shape

(18285, 5)

In [ ]:
## Get the Independent Features
X=df.drop('label',axis=1)

In [ ]:
## Get the Dependent features
y=df['label']

In [ ]:
X.shape

(18285, 4)

In [ ]:
y.shape

(18285,)

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
### Vocabulary size
voc_size=5000

### Onehot

In [ ]:
messages=X.copy()

In [ ]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
### Dataset Preprocessing
## Tokenization
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):

    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[4409, 4719, 2165, 1682, 834, 3000, 4847, 3519, 2250, 564],
 [1303, 2914, 585, 3638, 1767, 1377, 105],
 [4132, 3049, 426, 1389],
 [3832, 3611, 4219, 3154, 2450, 4602],
 [2721, 1767, 812, 305, 1417, 161, 1767, 456, 2135, 1551],
 [987,
  605,
  4484,
  4710,
  1290,
  4697,
  524,
  3971,
  1281,
  1027,
  784,
  2830,
  881,
  1417,
  105],
 [2392, 468, 4340, 1776, 932, 3029, 938, 4793, 4901, 3739, 3329],
 [2897, 3331, 1510, 1550, 329, 1821, 4697, 217, 4901, 3739, 3329],
 [1036, 381, 959, 1159, 3470, 159, 2409, 1788, 4697, 1508],
 [4364, 2840, 613, 2957, 4083, 17, 2736, 3745],
 [217, 3797, 2405, 4290, 2009, 4473, 271, 1404, 3707, 4510, 567],
 [3154, 306, 834, 159, 4697, 329],
 [3862, 1608, 941, 411, 4340, 884, 172, 3890, 1240],
 [4858, 2960, 518, 4808, 3665, 1853, 3952, 4901, 3739, 3329],
 [1538, 4399, 3737, 1226, 1286, 4901, 3739, 3329],
 [3161, 983, 4077, 4771, 2755, 1708, 435, 1028, 2420, 1536],
 [4736, 4457, 2914],
 [4267, 4505, 4567, 724, 4697, 2050, 1212, 105],
 [1359, 988, 585, 

In [ ]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [ ]:
onehot_repr[1]

[1303, 2914, 585, 3638, 1767, 1377, 105]

### Embedding Representation

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)
#It creates a uniform matrix of shape (num_samples, 20)

[[4409 4719 2165 ...    0    0    0]
 [1303 2914  585 ...    0    0    0]
 [4132 3049  426 ...    0    0    0]
 ...
 [1616  653 3135 ...    0    0    0]
 [3653  329 3249 ...    0    0    0]
 [2782 3444 3487 ...    0    0    0]]


In [ ]:
embedded_docs[1]

array([1303, 2914,  585, 3638, 1767, 1377,  105,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [ ]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(input_dim=voc_size,output_dim=embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
len(embedded_docs),y.shape

(18285, (18285,))

In [ ]:
#SPLIT

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=12,batch_size=64)

Epoch 1/12
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.7647 - loss: 0.4296 - val_accuracy: 0.9170 - val_loss: 0.1944
Epoch 2/12
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9473 - loss: 0.1362 - val_accuracy: 0.9176 - val_loss: 0.2132
Epoch 3/12
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9660 - loss: 0.0904 - val_accuracy: 0.9152 - val_loss: 0.2173
Epoch 4/12
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9756 - loss: 0.0719 - val_accuracy: 0.9112 - val_loss: 0.3354
Epoch 5/12
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9851 - loss: 0.0472 - val_accuracy: 0.9070 - val_loss: 0.4386
Epoch 6/12
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9902 - loss: 0.0329 - val_accuracy: 0.9117 - val_loss: 0.3580
Epoch 7/12
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9905 - loss: 0.0318 - val_accuracy: 0.9049 - val_loss: 0.4565
Epoch 8/12
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9928 - loss: 0.0267 - val_accuracy: 0

### Adding Dropout

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


### Performance Metrics And Accuracy

In [ ]:
y_pred=model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [ ]:
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

array([[3107,  312],
       [ 226, 2390]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9108533554266777

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.88      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

